In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys

sys.path.append("../")

In [20]:
from statapp.transformer.tensorflow.tf_model import *

### Preprocessing

In [22]:
DATA = "/home/moog/statapp_language_model/data/fr.train.top1M.txt"
train, test, val, encoder = load_train_test_val_encoder(data=DATA, sample=2E-5)
vocab_size = encoder.vocab_size - 1

X_train, y_train = split_into_X_y(train, hparams["seq_length"], one_hot_encode_y=True, vocab_size=vocab_size)
X_test, y_test = split_into_X_y(test, hparams["seq_length"], one_hot_encode_y=True, vocab_size=vocab_size)
X_val, y_val = split_into_X_y(val, hparams["seq_length"], one_hot_encode_y=True, vocab_size=vocab_size)

### Instantiating a Transformer

In [35]:
print("The hyperparameters are :\n", hparams)

The hyperparameters are :
 {'seq_length': 32, 'num_blocks': 4, 'd_model': 64, 'num_heads': 8, 'target_vocab_size': 1000, 'epochs': 1, 'batch_size': 256, 'learning_rate': 0.001}


In [27]:
# Form model
model = build_transformer(vocab_size=vocab_size)
# from_logits: Whether y_pred is expected to be a logits tensor
model.compile(
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(hparams["learning_rate"]),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

SHAPE Tensor("Shape_2:0", shape=(2,), dtype=int32)


In [28]:
summary = []
model.summary(print_fn=lambda x: summary.append(x))
summary = "\n".join(summary)
print(summary)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 32, 64)            16384     
_________________________________________________________________
tf_op_layer_positional_encod [(None, 32, 64)]          0         
_________________________________________________________________
encoder_block_8 (EncoderBloc (None, 32, 64)            16896     
_________________________________________________________________
encoder_block_9 (EncoderBloc (None, 32, 64)            16896     
_________________________________________________________________
encoder_block_10 (EncoderBlo (None, 32, 64)            16896     
_________________________________________________________________
encoder_block_11 (EncoderBlo (None, 32, 64)            1689

### Train

In [32]:
history = model.fit(
    X_train,
    y_train,
    # steps_per_epoch=np.ceil(len(X_train)/batch_size),
    batch_size=hparams["batch_size"],
    epochs=hparams["epochs"],
    validation_data=(X_val, y_val),
)

Train on 293 samples, validate on 108 samples
293/293 [==============================] - 2s 7ms/sample - loss: 4.0594 - categorical_accuracy: 0.1911 - val_loss: 3.9579 - val_categorical_accuracy: 0.1667


### Compute perplexity

In [33]:
#perp = calculate_perplexity(model, X_test, y_test)

### Prediction

In [34]:
prompt = "Il y a bien longtemps , dans un pays lointain , "
generated_text = generate_sampled(model, encoder, hparams["seq_length"], 500, prompt, 1)
print(generated_text)

  0%|          | 0/500 [00:00<?, ?it/s]

Generating sampled...


100%|██████████| 500/500 [00:24<00:00, 20.56it/s]

Texte généré
Il y a bien longtemps , dans un pays lointain , f{!�w!!!����D!uwx�pgt!pgff!!>!fjQ!g��j�!f!!!x!wxt!!��2!xnX_f�!!yf�x�!�j!w�����L'�g!t�f!5�8!�f!x�b��N^��!fi!!!!�!p�!2fNsfffWf��f.!2pn!!�{q!�!s!�y��tf!pgf!�j!!!!�qj�f8!yf��!ff=f!!!!�![2gpf!+�!�t�=Vg�!!w!!�!j'!!��!!�!!a�u%gq!!x!f��"!�f1L�%fx!!!!!�x�ffut!f!ffff!!�fs	sf!�[jx߾!!�!�!!Nf8s!ff!��!����!�f�![f!׳f!tx�!f!�!�!f!uu!�!��"et!߾x!!�!!!a!�p�ff�m�![J!�tfo�!�tft!g�b�!!
b!!�"uf!tf[u�!hf�!��fp��!��!t!uf�!t!��!f�!�pt�!ftf!f!f!jsj!!ftt��!!!�fo!!�!��+!n��bf!�!�


### Append to log

In [ ]:
"""
log = {
    "hyperparameters": hparams,
    "history": history.history,
    "summary": summary,
    "sample": {
        "prompt": prompt,
        "output": generated_text[len(prompt):],
    },
    "metrics": {
        "perplexity": perp,
    },
    "data_size": {
        "train": len(X_train),
        "val": len(X_val),
        "test": len(X_test),
    },
    "comment": comment,
}

if log_training:
    add_to_log(log)

pprint(log, compact=True)
"""